<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-computation-and-storage" data-toc-modified-id="Data-computation-and-storage-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data computation and storage</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analysis</a></span></li></ul></div>

In [1]:
# scientific package imports
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd
import plotly.express as px

import astropy.units as u

from psycopg2.extras import execute_values

# import my own classes
# from galaxy.galaxy import Galaxy
# from galaxy.galaxies import Galaxies
# from galaxy.massprofile import MassProfile
from galaxy.timecourse import TimeCourse
# from galaxy.centerofmass import CenterOfMass
# from galaxy.plots import Plots
from galaxy.db import DB

In [2]:
db = DB()
tc = TimeCourse()
cur = db.get_cursor()

## Data computation and storage

In [3]:
def get_distances(gal, snap):
    data = np.array(db.get_xyz(gal, snap),
                    dtype=[('pnum', '<i4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4')])
    xyz = np.array([data[xi] for xi in ('x','y','z')])
    MW_com, _ = tc.get_one_com('MW', snap)
    M31_com, _ = tc.get_one_com('M31', snap)
    M33_com, _ = tc.get_one_com('M33', snap)
    d_MW = norm(xyz - MW_com[:, np.newaxis], axis=0)
    d_M31 = norm(xyz - M31_com[:, np.newaxis], axis=0)
    d_M33 = norm(xyz - M33_com[:, np.newaxis], axis=0)
    return data['pnum'], d_MW, d_M31, d_M33

In [4]:
def write_dist_db(gal, snap):
    pnum, d_MW, d_M31, d_M33 = get_distances(gal, snap)
    dist = np.array([d_MW, d_M31, d_M33])
    nearest = np.argmin(dist, axis=0)
    
    data = [(gal, snap, int(pnum[i]), float(d_MW[i]), 
               float(d_M31[i]), float(d_M33[i]), int(nearest[i]))
                for i in range(len(pnum))]
#     return data

    colheads = ','.join(['galname','snap','pnum','d_mw','d_m31','d_m33','nearest'])
    query = f"""
        INSERT INTO distances( {colheads} ) 
        VALUES %s
        ON CONFLICT DO NOTHING
        """
    
    execute_values(cur, query, data, page_size=1000)

In [5]:
# for snap in range(10, 802, 10):
#     for gal in ('MW','M31','M33'):
#         print(gal, snap)
#         data = write_dist_db(gal, snap)

## Analysis

In [3]:
sql = """
SELECT d.galname, t.t, count(*)
    FROM public.distances d
    INNER JOIN public.totalcom t ON d.snap=t.snap
    WHERE (d.galname='MW' and d.nearest != 0)
        OR (d.galname='M31' and d.nearest != 1)
        OR (d.galname='M33' and d.nearest != 2)
    GROUP BY d.galname, t.t
    ORDER BY d.galname, t.t;
"""

transfers = db.run_query(sql)

In [5]:
df = pd.DataFrame(transfers, columns=['galaxy','time (Gyr)','count'])
df.to_csv('transfers.csv', index=False)
df.head()

,galaxy,time (Gyr),count
0,M31,0.000,112752
1,M31,0.143,60050
2,M31,0.286,65519
3,M31,0.429,72603
4,M31,0.571,80426


In [42]:
fig = px.line(df, x='time (Gyr)', y='count', color='galaxy')
fig.show()

In [13]:
# DO NOT ask for too many snaps at once. 
# The intermediate query gets ridiculously large and runs out of disk/memory

sql2 = """
SELECT d.galname, s.time/1000, s.type, count(*)
    FROM public.distances d
    INNER JOIN public.simdata s 
    ON d.snap=s.snap AND d.galname=s.galname AND d.pnum=s.pnum
    WHERE ((d.galname='MW' and d.nearest != 0)
        OR (d.galname='M31' and d.nearest != 1)
        OR (d.galname='M33' and d.nearest != 2))
    AND d.snap in (0, 100, 200, 277, 335, 400)
    GROUP BY d.galname, s.time, s.type
    ORDER BY d.galname, s.time, s.type;
"""

tr_ptype = db.run_query(sql2)

In [17]:
df2 = pd.DataFrame(tr_ptype, columns=['galaxy','time (Gyr)','ptype','count'])
df2.head()

,galaxy,time (Gyr),ptype,count
0,M31,0.00000,1,111926
1,M31,0.00000,3,826
2,M31,1.42857,1,71757
3,M31,1.42857,3,715
4,M31,2.85714,1,61687


In [18]:
types = {1: 'Halo', 2: 'Disk', 3: 'Bulge'}
df2['Particle type'] = df2['ptype'].map(types)
df2.to_csv('transfers_ptype.csv', index=False)
df2.head()

,galaxy,time (Gyr),ptype,count,Particle type
0,M31,0.00000,1,111926,Halo
1,M31,0.00000,3,826,Bulge
2,M31,1.42857,1,71757,Halo
3,M31,1.42857,3,715,Bulge
4,M31,2.85714,1,61687,Halo


In [16]:
import plotly.graph_objects as go

d = px.bar(df2, x='time (Gyr)', y='count', color='Particle type', facet_col="galaxy")
# d.update_layout(uniformtext_minsize=20, uniformtext_mode='hide')
d.update_layout(
#     title="Transferred particles by type",
    font=dict(
        size=20,
#         color="#7f7f7f"
    )
)

d.show()